In [45]:
from twnews.wtmf.support_functions import memo_process, dump, load
from twnews.dataset.dataset import Dataset
from twnews.timeit import timeit

texts = load('texts')
#P,Q = load('PQ')
dataset = Dataset(fraction=1, use_dataset_if_exist=True)
news_texts = dataset.news_storage.get_texts()
news = dataset.news_storage.get_news()
tweets = dataset.dataset
tweets_texts = [tweet.text for tweet in dataset.dataset]

2016-04-08 01:32:58.886807: Start of loading news from /home/avybornov/twnews_data/rss
2016-04-08 01:32:59.511320: News successfully loaded


In [46]:

len(news), len(tweets)

(7084, 1091)

In [47]:
nt = []

for i in range(len(news_texts)):
    nt.append((news[i], Q[:,i]))

tt = []
for i in range(len(tweets_texts)):
    tt.append((tweets[i], Q[:,i+len(news_texts)]))


In [48]:
from sklearn.metrics.pairwise import cosine_similarity

    #sorted(sim_list, key=lambda x: x[1])
    

In [49]:
from multiprocessing import Pool

def find_topk_sim_news_to_tweet(tweet_pair, news_pairs, k=10):
    tweet, tweet_latent_vector = tweet_pair
    sim_list = [(x[0],cosine_similarity(tweet_latent_vector, x[1])[0][0]) for x in news_pairs]
    sim_list = sorted(sim_list, key=lambda x: x[1], reverse=True)
    #print map(lambda x: x[1], sim_list[:10])
    return map(lambda x: x[0], sim_list[:k])

def f(x):
    t, news_pairs, k = x
    res = find_topk_sim_news_to_tweet(t, news_pairs, k)
    return (t[0], res)

@timeit
def find_topk_sim_news_to_tweets(tweet_pairs, news_pairs, k=10):    
    top = []

    p = Pool(16)
    return p.map(f, [(tweet, news_pairs, k) for tweet in tweet_pairs])
    #return top

In [50]:
top = find_topk_sim_news_to_tweets(tt,nt, k=10)


2016-04-08 01:33:04.593133: Function find_topk_sim_news_to_tweets started with time measure
2016-04-08 01:37:51.887236: Function find_topk_sim_news_to_tweets finished in 4m47.2938511372s


In [53]:
def TOPK_t(tweet, top_news, k):
    top_news = top_news[:k]
    for news in top_news:
        for url in tweet.urls:
            if url == news.link:
                return 1
    return 0

def TOPK(tweets, k):
    s = 0
    for tweet, top_news in tweets:
        s += TOPK_t(tweet, top_news, k)
    return s*1.0/len(tweets)


def ATOP(tweets, N):
    s = 0
    for k in range(1,N+1):
        s += TOPK(tweets, k)
    return s*1.0/N

# for res in top[:2]:
#     tweet, top_news = res
#     print tweet, 
#     for news in top_news:
#         print news.title
#     print '-------'
ATOP(top,10)
    #top_news
#print top[0][1][0].title


0.27277726856095325

In [58]:
import random
k = 10
set_news = set(news)
random_top = []
for tweet in tweets:
    sample = random.sample(set_news, k)
    random_top.append((tweet, sample))

In [59]:
ATOP(random_top,10)

0.0009165902841429881